### Testing API

In [21]:
# base_url = 'https://api.pushshift.io/reddit/'


# def request_posts(subreddit, days_ago, base_url=base_url,
#                   endpoint='search/submission/', is_video='is_video=false'):
#     try:
#         response = requests.get(
#             f'{base_url}{endpoint}?subreddit={subreddit}&{is_video}&before={days_ago}d&after={days_ago+1}d&size=1000')
#         assert response.status_code == 200
#     except:
#         pass

#     return response


# def make_requests(subreddit, days_of_data):
#     all_results = []

#     for i in range(1, days_of_data):
#         try:
#             entry = request_posts(subreddit, i)
#             all_results.append(pd.DataFrame(entry.json()['data']))
#         except:
#             pass
#         if i % 20 == 0:
#             print(f'{i} of {days_of_data} requests completed')
#         time.sleep(1.5)

#     return pd.concat(all_results)


# def request_all_subs(list_of_subreddits, days_of_data):
#     all_results = []
#     for sub in list_of_subreddits:
#         print(f'Querying {sub}...')
#         sub_df = make_requests(sub, days_of_data)
#         all_results.append(sub_df)
#     return pd.concat(all_results)


# def main():
#     subs = ['asoiaf', 'freefolk', 'gameofthrones']
#     df = request_all_subs(subs, 100)
#     df.to_csv('./data/subreddit_data.csv')


# if __name__ == "__main__":
#     main()

### Validating API results

In [22]:
# # This script accepts the raw API response .csv and outputs one with NAs and deleted posts removed,
# # plus any HTML fragments removed 

# # Imports
# from bs4 import BeautifulSoup
# import pandas as pd
# import warnings

# # Ignore warnings
# warnings.filterwarnings("ignore")

# # Import Data
# df = pd.read_csv('../data/subreddit_data.csv')

# print('Number of Entries per subreddit pre-cleaning:')
# print(df['subreddit'].value_counts())

# # Drop posts without selftext
# has_self = (df[~df['selftext'].isna()])
# has_self = has_self[has_self['domain'].str.contains('self')]

# # Drop posts where text has been removed or deleted by users/moderators
# has_self = has_self[has_self['selftext']!='[removed]']
# has_self = has_self[has_self['selftext']!='[deleted]']
# has_self = has_self[~has_self['selftext'].str.startswith('[removed]')]
# has_self = has_self[~has_self['selftext'].str.startswith('[deleted]')]

# # Select the columns we're interested in modeling
# columns = ['author', 'full_link', 'id', 'num_comments', 'num_crossposts', 'over_18',
#                      'spoiler', 'subreddit', 'title', 'total_awards_received', 'upvote_ratio',
#                      'edited', 'gilded','selftext']

# has_self = has_self[columns]

# # Reset index of results because it is annoying 
# has_self.reset_index(drop=True, inplace=True)

# print('Number of Entries per subreddit post-cleaning:')
# print(has_self['subreddit'].value_counts())

# # Just in case there are any HTML impurities in the raw data, we will use BeautifulSoup 
# # to clean the text columns
# for col in has_self:
#     if has_self[col].dtypes == 'O':
#         for item in has_self.index:
#             has_self.loc[item,col] = BeautifulSoup(has_self.loc[item,col]).get_text()

# has_self.to_csv('../data/cleaned_subreddit_data.csv', index=False)

### Investigating our full data

In [28]:
raw = pd.read_csv('../data/subreddit_data.csv')

C:\Users\Craig\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,4,6,7,8,9,10,11,16,18,19,20,21,22,25,27,28,30,31,33,34,40,45,51,55,60,63,64,65,66,67,71,72,73,78,79,81,82,84,86,89,90,91,92,94,97,98,100,101,102,103,104,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
columns = ['id','selftext','author','title','subreddit']

raw = raw[columns]

raw.dropna(inplace=True)

In [41]:
raw[raw['selftext'].str.contains("\xa0")]['selftext']

6         What is the Most and Least Knightly Acts in AS...
155        Recently I have completed a portion of my big...
322       Brienne. People always praise GRRM for writing...
358       A Storm of Sword was more interesting for me t...
419       There are lots of theories on the identity of ...
                                ...                        
234791    Strange thing with Sansa's wedding scene i spo...
234870    https://en.m.wikipedia.org/wiki/Damascus_steel...
237610    Why was Jon Snow given the name Snow?\n\nI tho...
241723    So, Joffrey is poisoned with a drink by Olenna...
242867    In season 3 episode 6 Melisandre says: "I see ...
Name: selftext, Length: 765, dtype: object

In [45]:
raw.loc[155:157,'selftext'].str.replace('\xa0',' ')

155     Recently I have completed a portion of my big...
Name: selftext, dtype: object

In [35]:
raw.loc[6,'selftext']

'What is the Most and Least Knightly Acts in ASOIAF in your opinion?\n\nMy most knightly act of the series is an ironic choice because the deed is done by Brienne of Tarth, who honored her vow to protect the weak, by defending the orphans from some of Westero\'s worst monsters. Compare her act to the Knight of Saltpans who sat in his castle, as "The Hound" ravages the town. Brienne is the definition of a knight. A close second is a minor, yet gallant act. During the wedding between Sansa Stark and Tyrion Lannister, Ser Garland is one of\xa0the few people who is actually nice to Sansa, and his dance with the terrified, young maid is the highlight of her night. Ser Garland is similarly kind to Tyrion, and goes so far to defend him in front of the Incest King.\n\nMy least knightly act of the series, would probably be Ser Gregor\'s brutal campaign throughout the Riverlands, which featured such atrocities as rape, torture, and forced auto-cannibalism. Compare this with Ser Arthur\'s Dayne c

In [31]:
raw.loc[28,'selftext']

'&gt;If  I had been a better knight… if I had unhorsed the prince in that last  tilt, as I unhorsed so many others, it would have been for me to choose  the queen of love and beauty…\n\nBarristan lost to Rhaegar during the tourney at Harrenhal:\n\n&gt;Prince  Rhaegar emerged as the ultimate victor at the end of the  competition.  The crown prince, who did not normally compete in tourneys,  surprised  all by donning his armor and defeating every foe he faced, including  four knights of the Kingsguard. In the final tilt, he unhorsed Ser  Barristan Selmy, generally regarded as the finest lance in all the Seven  Kingdoms, to win the champion\'s laurels.\n\nSo  Rhaegar, who "did not normally compete in tourneys" manages to beat  "the finest lance in the realm". Not only that, but he also defeats  three other members of the Kingsguard (possibly Arthur Dayne, Lewyn  Martell, and Oswell Whent?).\n\nBarristan  doesn\'t say "If I had been a better jouster", he says he should have  been "a better

In [33]:
raw.loc[28:29,'selftext'].str.replace('&gt;',' ')

28     If  I had been a better knight… if I had unho...
29    Since I have PTSD from the Red Wedding, I’m th...
Name: selftext, dtype: object

In [46]:
#from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from scipy.stats import uniform, loguniform
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#import warnings

In [2]:
df = pd.read_csv('../data/preprocessed_subreddit_data.csv')

In [3]:
df.shape

(70985, 7)

In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(70985, 7)

In [6]:
df.selftext[0]

"['For', 'me', 'it', 'is', 'Jon', 'Snow', 'He', 'used', 'to', 'be', 'my', 'favorite', 'but', 'the', 'overpandering', 'to', 'his', 'character', 'by', 'saying', 'he', 'is', 'the', 'subject', 'to', 'every', 'prophecy', 'and', 'is', 'basically', 'Jesus', 'is', 'so', 'annoying', 'He', 'is', 'still', 'a', 'good', 'character', 'especially', 'in', 'the', 'book', 'but', 'thinking', 'jon', 'snow', 'will', 'defeat', 'the', 'other', 'then', 'he', 'will', 'kill', 'dany', 'and', 'then', 'rule', 'wisely', 'and', 'justly', 'because', 'he', 'is', 'the', 'perfect', 'hero', 'is', 'annoying']"

In [7]:
# df = df.drop(df[df['author'] == 'AutoModerator'].index)

#  df.loc[:,'selftext'] = df['selftext'].str.replace('x200B','')

#  df.loc[:,'selftext'] = df['selftext'].str.replace('\n',' ')

# df = df[~df['selftext'].isna()]

# df.reset_index(inplace=True)

In [8]:
df.columns

Index(['level_0', 'index', 'id', 'selftext', 'author', 'title', 'subreddit'], dtype='object')

In [9]:
df.shape

(70985, 7)

In [10]:
# most_common_authors = df.author.value_counts().sort_values(ascending=False).head(21).index[1:]

# table = df.loc[df['author'].isin(most_common_authors),['author','subreddit','id']].groupby(['author','subreddit']).count()

# table.index.levels

# ax = table.unstack().plot.barh(figsize=(10,10))


In [17]:
df[df['selftext'].str.contains("gt")]['selftext']


8        ['As', 'you', 'know', 'GRRM', 'made', 'a', 'st...
11       ['gtIf', 'I', 'had', 'been', 'a', 'better', 'k...
12       ['Since', 'I', 'have', 'PTSD', 'from', 'the', ...
13       ['From', 'JonCon', 'in', 'ADWD', 'gt', 'He', '...
15       ['There', 'is', 'a', 'number', 'of', 'fact', '...
                               ...                        
70869    ['Long', 'post', 'Lets', 'walk', 'through', 's...
70872    ['When', 'i', 'saw', 'the', 'scene', 'with', '...
70899    ['Look', 'I', 'know', 'nobody', 'liked', 'Ella...
70900    ['Ive', 'seen', 'quite', 'a', 'few', 'people',...
70913    ['John', 'left', 'Winterfell', 'and', 'everyon...
Name: selftext, Length: 7702, dtype: object

In [18]:
df.loc[11,'selftext']

"['gtIf', 'I', 'had', 'been', 'a', 'better', 'knight…', 'if', 'I', 'had', 'unhorsed', 'the', 'prince', 'in', 'that', 'last', 'tilt', 'a', 'I', 'unhorsed', 'so', 'many', 'others', 'it', 'would', 'have', 'been', 'for', 'me', 'to', 'choose', 'the', 'queen', 'of', 'love', 'and', 'beauty…', 'Barristan', 'lost', 'to', 'Rhaegar', 'during', 'the', 'tourney', 'at', 'Harrenhal', 'gtPrince', 'Rhaegar', 'emerged', 'a', 'the', 'ultimate', 'victor', 'at', 'the', 'end', 'of', 'the', 'competition', 'The', 'crown', 'prince', 'who', 'did', 'not', 'normally', 'compete', 'in', 'tourney', 'surprised', 'all', 'by', 'donning', 'his', 'armor', 'and', 'defeating', 'every', 'foe', 'he', 'faced', 'including', 'four', 'knight', 'of', 'the', 'Kingsguard', 'In', 'the', 'final', 'tilt', 'he', 'unhorsed', 'Ser', 'Barristan', 'Selmy', 'generally', 'regarded', 'a', 'the', 'finest', 'lance', 'in', 'all', 'the', 'Seven', 'Kingdoms', 'to', 'win', 'the', 'champion', 'laurel', 'So', 'Rhaegar', 'who', 'did', 'not', 'normally

In [19]:
df.loc[15,'selftext']

"['There', 'is', 'a', 'number', 'of', 'fact', 'we', 'can', 'be', 'sure', 'of', 'concerning', 'Danys', 'origin', 'Fact', '1', 'Rhaella', 'gave', 'birth', 'to', 'a', 'girl', 'named', 'Daenerys', 'on', 'Dragonstone', 'How', 'can', 'we', 'certain', 'of', 'that', 'Because', 'of', 'this', 'statement', 'amp', 'gt', 'That', 'may', 'be', 'Or', 'not', 'Kevan', 'Lannister', 'had', 'been', 'here', 'in', 'this', 'very', 'hall', 'when', 'Tywin', 'had', 'laid', 'the', 'body', 'of', 'Prince', 'Rhaegars', 'child', 'at', 'the', 'foot', 'of', 'the', 'Iron', 'Throne', 'wrapped', 'up', 'in', 'crimson', 'cloak', 'The', 'girl', 'had', 'been', 'recognizably', 'the', 'Princess', 'Rhaenys', 'but', 'the', 'boy', '…', 'a', 'faceless', 'horror', 'of', 'bone', 'and', 'brain', 'and', 'gore', 'a', 'few', 'hank', 'of', 'fair', 'hair', 'None', 'of', 'u', 'looked', 'long', 'Tywin', 'said', 'that', 'it', 'wa', 'Prince', 'Aegon', 'and', 'we', 'took', 'him', 'at', 'his', 'word', 'We', 'have', 'these', 'tale', 'coming', 'fr

In [11]:
tfid_reddit = TfidfVectorizer(lowercase=True, stop_words='english',
                              ngram_range=(1,2),max_df=.95, min_df=3, max_features=5000)

vect_table = tfid_reddit.fit_transform(df['selftext'])

vect_df = pd.DataFrame(vect_table.toarray(), columns = tfid_reddit.get_feature_names())

vect_df_sorted = vect_df.sum().sort_values()

In [12]:
vect_df_sorted.head(100)

door hold         2.418168
gt arya           6.544665
serra             6.920188
lewyn             7.282225
ampnbsp house     7.289751
black eye         7.451584
xii               7.573839
elder brother     7.775377
gtbran            8.456489
shadrich          8.874994
gtbut             8.912504
daenerys iv       8.959781
gt tyrion         9.019384
agot gt           9.047674
gtas              9.072678
mouse             9.149530
hosteen           9.205317
prey              9.255739
hypothesis        9.282239
gtmy              9.287901
amethyst          9.358661
xi                9.385269
sealord           9.585352
gtlord            9.696330
asharas           9.774302
brow              9.789419
informs           9.834322
aeryss            9.932858
gthis             9.958318
oak               9.964327
bran iii          9.966074
cok              10.024826
echo             10.097865
robe             10.449931
gtser            10.586942
ty               10.597682
vanished         10.599697
g

In [13]:
vect_df_sorted.tail(100)

seen           471.516653
world          472.115490
army           477.671160
die            480.010692
hand           481.007562
look           486.655323
ending         486.723232
killed         489.722943
snow           492.211262
life           493.072110
grrm           494.392524
thats          496.473927
kings          496.990277
actually       499.432697
war            506.915709
new            510.390938
targaryen      512.961710
men            515.155005
lot            520.246303
point          525.676389
stannis        530.787598
great          530.937690
man            532.250413
white          536.020936
doesnt         536.520689
guy            540.916167
long           543.657526
right          546.083823
night king     565.276823
feel           570.033149
mean           574.035884
dead           576.166780
battle         578.298237
north          581.082718
winterfell     582.842557
westeros       585.101063
watch          586.700209
gt             586.743784
child       

In [ ]:
vect_df['subreddit'] = df['subreddit']
vect_df['selftext'] = df['selftext']

vect_df[vect_df['subreddit'] == 'freefolk'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

In [ ]:
vect_df[vect_df['subreddit'] == 'gameofthrones'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

In [ ]:
vect_df[vect_df['subreddit'] == 'asoiaf'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

In [34]:
df.loc[45,'selftext']

'Like yes I know you get Theons internal monologues and such is the book but I feel hes a lot more sympathetic and emotional in the show Alfie Allen does a fantastic job and you really see the conflict in his acting and his regret for what hes done Maester Luwin too i feel is a lot better and Rodriks death is definitely more impactful in the show than the book'

In [35]:
df['selftext'].isna().sum()

0

In [36]:
df['title'].isna().sum()

0

Preprocessing workflow:

- tokenize
- remove stopwords
- lemmatize


In [37]:
df['selftext'] = [word_tokenize(str(text)) for text in df['selftext']]

df['title'] = [word_tokenize(str(text)) for text in df['title']]

In [38]:
lemmatizer = WordNetLemmatizer()

In [39]:
lemmatizer.lemmatize('churches')

'church'

In [40]:
df['selftext'] = [[lemmatizer.lemmatize(word) for word in text] for text in df['selftext']]

In [41]:
stemmer = PorterStemmer()

In [42]:
df['selftext'] = [' '.join([stemmer.stem(word) for word in text]) for text in df['selftext']]

In [43]:
df.selftext[0]

'for me it is jon snow He use to be my favorit but the overpand to hi charact by say he is the subject to everi propheci and is basic jesu is so annoy He is still a good charact especi in the book but think jon snow will defeat the other then he will kill dani and then rule wise and justli becaus he is the perfect hero is annoy'

### Testing Modeling process

In [44]:
sample = df.sample(frac=.05)

sample.subreddit.value_counts()

sample.subreddit.value_counts(normalize=True)

sample.shape

X = sample['selftext']
y = sample['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

In [49]:
# tfid_final_params = {'tfid__max_df': 0.325,
#   'tfid__max_features': 6500,
#   'tfid__min_df': 5,
#   'tfid__ngram_range': (1, 2)}


#### Baseline independent models: LogisticRegression and DecisionTree

##### Logistic Regression

In [136]:
logit_pipe = Pipeline([
    ('tfid',TfidfVectorizer(max_df=.325,
                           max_features = 6500,
                           min_df = 5,
                           ngram_range = (1,2))),
    ('logit',LogisticRegression(solver='liblinear'))
])

logit_params = {
    'logit__C':loguniform(.01,50),
    'logit__penalty':['l2','l1',None],
}

In [137]:
logit_model = RandomizedSearchCV(logit_pipe,
                  logit_params,
                  n_iter = 75,
                  n_jobs=-1,
                  cv = 3,
                  verbose=1)

In [138]:
logit_model.fit(X_train,y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('tfid',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=0.325,
                                                              max_features=6500,
                                                              min_df=5,
                                                      

In [139]:
logit_model.best_params_

{'logit__C': 2.552049865944663, 'logit__penalty': 'l2'}

In [140]:
cross_val_score(logit_model.best_estimator_,X_train,y_train, cv=5)

array([0.56554307, 0.58988764, 0.57677903, 0.57035647, 0.57973734])

In [141]:
cross_val_score(logit_model.best_estimator_,X_test,y_test, cv=5)

array([0.5505618 , 0.61797753, 0.61235955, 0.58988764, 0.53370787])

In [142]:
logit_model.score(X_test,y_test) - logit_model.score(X_train,y_train)

-0.2984409480652932

In [143]:
logit_model.best_estimator_.score(X_test,y_test) - logit_model.best_estimator_.score(X_train,y_train)

-0.2984409480652932

In [144]:
logit_model.best_estimator_.score(X_train,y_train)

0.9186656671664168

In [145]:
logit_model.best_estimator_.score(X_test,y_test)

0.6202247191011236

##### Decision Tree

In [111]:
tree_pipe = Pipeline([
    ('tfid',TfidfVectorizer(max_df=.325,
                           max_features = 6500,
                           min_df = 5,
                           ngram_range = (1,2))),
    ('tree',DecisionTreeClassifier())
])

tree_params = {
    'tree__max_depth':uniform(1,20),
    'tree__min_samples_split':uniform(0,.5),
    'tree__min_samples_leaf':uniform(0,.5),
}

In [112]:
tree_model = RandomizedSearchCV(tree_pipe,
                  tree_params,
                  n_iter = 100,
                  n_jobs=-1,
                  cv = 3,
                  verbose=1)

In [113]:
tree_model.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('tfid',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=0.325,
                                                              max_features=6500,
                                                              min_df=5,
                                                      

In [114]:
tree_model.best_params_

{'tree__max_depth': 18.788855431067592,
 'tree__min_samples_leaf': 0.0064049947690571485,
 'tree__min_samples_split': 0.33934451621277056}

In [115]:
cross_val_score(tree_model.best_estimator_,X_train,y_train, cv=5)

array([0.47565543, 0.52621723, 0.52059925, 0.50281426, 0.48405253])

In [116]:
cross_val_score(tree_model.best_estimator_,X_test,y_test, cv=5)

array([0.44382022, 0.47752809, 0.52808989, 0.43820225, 0.41573034])

In [117]:
tree_model.score(X_test,y_test) - tree_model.score(X_train,y_train)

-0.057350538214039015

In [119]:
tree_model.best_estimator_.score(X_test,y_test) - tree_model.best_estimator_.score(X_train,y_train)

-0.057350538214039015

In [123]:
tree_model.best_estimator_.score(X_train,y_train)

0.5629685157421289

In [124]:
tree_model.best_estimator_.score(X_test,y_test)

0.5056179775280899

#### Next step: implement voting classifier

In [ ]:
#Voting Classifier:
# LogisticRegression
# MultinomialNB
# GradientBoostingClassifier
# AdaBoostClassifier
# RandomForestClassifier
# DecisionTreeClassifier
# SVC


# Pipeline (tfid => Voting Classifier)

# Random 

5.869304461753987